# Iteration 1

## 1 Aim:

The aim of the first iteration is to build an intitial simulation model of a critical care unit (CCU), considering patients from unplanned admissions only.

## 2 Prompt:

In the paper published by Griffiths et al., the simulation model of CCU admits patients from two different routes: `"unplanned admission"` and `"planned admission"`. In the former route, patients from five sources (Accident and Emergency, Ward, Emergency, Other hospitals, and X-Ray) have to wait until a bed is available, then they can enter the unit and stay here for a specified period of time to receive therapy. After the treatment is completed, the patient will leave the system immediately. The distributions of inter-arrival time (IAT) and length of stay (LOS), among the unplanned admitted patients are detailed in Table 1. In the first iteration, we will build the model based on the information of unplanned admissions, and obtain the respective metrics, such as mean number of occupied beds and the occupancy rate.

&nbsp;

<div style="text-align: center;"><em>Table 1. IAT and LOS distributions for unplanned admission patients. </em></div>


| Activity 	| Source        	| Distribution 	| Mean (hours) 	| Standard Dev (hours) 	|
| ---------	| ----------------- | -------------	| -------------	| --------------------- |
| IAT       |  A&E              | Exponential  	| 22.72         |                       |
| IAT       |  Ward             | Exponential  	| 26.0          |                       |
| IAT       |  Emergency        | Exponential  	| 37.0          |                       |
| IAT       |  Other hospitals  | Exponential  	| 47.2          |                       |
| IAT       |  X-Ray            | Exponential  	| 575.0         |                       |
| LOS       |  A&E              | Lognormal  	| 128.79        |  267.51               |
| LOS       |  Ward             | Lognormal  	| 177.89        |  276.54               |
| LOS       |  Emergency        | Lognormal  	| 140.15        |  218.02               |
| LOS       |  Other hospitals  | Lognormal  	| 212.86        |  457.67               |
| LOS       |  X-Ray            | Lognormal  	| 87.53         |  108.15               |



&nbsp;

Based on this description, the prompt inputed to the chatbox is as follows:

> ### Main command
Code a discrete-event simulation model of a critical care unit (CCU) in python 3.10 and simpy 4.
Please code the full model specified. Do not return a simplified version. Show all code.
The generated code does not need to be sectioned as the numbered sections in the prompt.
> 
> ### 1 General model logic
> > 1.1 All time units in the model are in hours.
> >
> > 1.2 Patients to arrive at the CCU from five different sources: Accident and Emergency (named as ae), Ward (named as ward), Emergency (named as emer), other hospitals (named as oth), or X-Ray department (named as xray). Each source had a different inter-arrival time (IAT) and length of stay (LOS) distribution.
All patient types must have their own generator function, for instance: def ae_arrivals_generator().
> >
> > 1.3 Each patient in the model has a unique identifier. The first patient to arrive has an identifier of 0. For each subsequent patient increment the identifier by 1.
> >
> > 1.4 There are 24 beds in the CCU.
> >
> > 1.5 After patients arrive, they occupy one bed for specific length of stay (LOS) if beds are available, and they leave the model afterwards.
> >
> > 1.6 The model should print out messages of patient arrive time, waiting time, LOS, and the time they leave.
> >
> ### 2 Simulation parameters
> * RUN_LENGTH = 12 * 30 * 24 (12 months)
> * N_BED = 24
> 
> 2.1 IAT
> > For each distribution time is measured in hours.
> > * Accident and Emergency (ae) = Exponential; Mean: 22.72
> > * Ward (ward) = Exponential; Mean: 26.0
> > * Emergency (emer) = Exponential; Mean: 37.0
> > * Other hospitals (oth) = Exponential; Mean: 47.2
> > * X-Ray (xray) = Exponential; Mean: 575.0
> >
> 2.2 LOS
> >  For each distribution time is measured in hours.
> > * Accident and Emergency (ae) = Lognormal; Mean: 128.79, std: 267.51
> > * Ward (ward) = Lognormal; Mean: 177.89, std: 276.54
> > * Emergency (emer) = Lognormal; Mean: 140.15, std: 218.02
> > * Other hospitals (oth) = Lognormal; Mean: 212.86, std: 457.67
> > * X-Ray (xray) = Lognormal; Mean: 87.53, std: 108.15
> ### 3 Simulation methodology
> > The numpy package should be used for sampling.
> > 
> > The itertools module should be used to generate the patient identifier.
> > 
> > Each IAT and LOS distribution should have its own numpy.random.Generator object.
> > 
> > Two classes Exponential and Lognormal have been created, with the sample() function to directly sample for the distributions. To call the function, use code like this: ae_arrival_dist = Exponential(mean, random_seed=seed).


## 3 LLM answer:

The code and text below were generated by the LLM model, i.e. ChatGPT:

In [15]:
# if not installed yet, run the code below
# !pip install simpy

In [27]:
'''
Creating a discrete-event simulation model of a Critical Care Unit (CCU) 
with the given specifications involves using simpy, a process-based 
discrete-event simulation framework based on standard Python. 

This simulation will track patient arrivals from different sources, 
their stay in the CCU, and their departure, 
all while managing a limited number of beds.

The model will be developed according to the given specifications, 
including the use of numpy for sampling from distributions, 
itertools for generating patient identifiers, 
and custom classes for the exponential and lognormal distributions. 
Note that due to the complexity of this model, the code will be quite extensive.

Let's start by coding the simulation model:
'''

import simpy
import numpy as np
import itertools
from scipy.stats import lognorm

# Custom classes for distributions
class Exponential:
    def __init__(self, mean, random_seed=None):
        self.mean = mean
        self.rng = np.random.default_rng(seed=random_seed)
    
    def sample(self):
        return self.rng.exponential(self.mean)

class Lognormal:
    def __init__(self, mean, std, random_seed=None):
        self.mean = mean
        self.std = std
        self.rng = np.random.default_rng(seed=random_seed)
        # Convert mean and std dev to the shape and scale that scipy expects
        self.sigma = np.sqrt(np.log(1 + (std/mean)**2))
        self.scale = np.exp(np.log(mean) - 0.5*self.sigma**2)
    
    def sample(self):
        return self.rng.lognormal(np.log(self.scale), self.sigma)

# Simulation parameters
RUN_LENGTH = 12 * 30 * 24  # 12 months
N_BEDS = 24

# Patient arrival source distributions
IAT_DISTRIBUTIONS = {
    "ae": Exponential(22.72),
    "ward": Exponential(26.0),
    "emer": Exponential(37.0),
    "oth": Exponential(47.2),
    "xray": Exponential(575.0)
}

# Patient length of stay (LOS) distributions
LOS_DISTRIBUTIONS = {
    "ae": Lognormal(128.79, 267.51),
    "ward": Lognormal(177.89, 276.54),
    "emer": Lognormal(140.15, 218.02),
    "oth": Lognormal(212.86, 457.67),
    "xray": Lognormal(87.53, 108.15)
}

# Generator for patient ids
patient_id_generator = itertools.count()

def patient_source(env, name, ccu, iat_dist, los_dist):
    while True:
        yield env.timeout(iat_dist.sample())
        patient_id = next(patient_id_generator)
        env.process(patient(env, patient_id, name, ccu, los_dist))

def patient(env, patient_id, source, ccu, los_dist):
    arrive_time = env.now
    print(f"Patient {patient_id} from {source} arrived at {arrive_time:.2f}")
    
    with ccu.request() as request:
        yield request
        wait_time = env.now - arrive_time
        los = los_dist.sample()
        print(f"Patient {patient_id} from {source} waited for {wait_time:.2f} hours, LOS: {los:.2f}")
        yield env.timeout(los)
        print(f"Patient {patient_id} from {source} left at {env.now:.2f}")

# Simulation environment
env = simpy.Environment()
ccu = simpy.Resource(env, capacity=N_BEDS)

# Start the patient sources
for source, iat_dist in IAT_DISTRIBUTIONS.items():
    los_dist = LOS_DISTRIBUTIONS[source]
    env.process(patient_source(env, source, ccu, iat_dist, los_dist))

# Run the simulation
env.run(until=RUN_LENGTH)


Patient 0 from ae arrived at 12.14
Patient 0 from ae waited for 0.00 hours, LOS: 8.38
Patient 1 from oth arrived at 13.94
Patient 1 from oth waited for 0.00 hours, LOS: 243.05
Patient 0 from ae left at 20.51
Patient 2 from ward arrived at 21.09
Patient 2 from ward waited for 0.00 hours, LOS: 17.84
Patient 2 from ward left at 38.92
Patient 3 from ae arrived at 49.48
Patient 3 from ae waited for 0.00 hours, LOS: 179.28
Patient 4 from emer arrived at 55.78
Patient 4 from emer waited for 0.00 hours, LOS: 168.32
Patient 5 from oth arrived at 56.96
Patient 5 from oth waited for 0.00 hours, LOS: 125.23
Patient 6 from oth arrived at 68.42
Patient 6 from oth waited for 0.00 hours, LOS: 50.22
Patient 7 from ward arrived at 70.00
Patient 7 from ward waited for 0.00 hours, LOS: 35.46
Patient 8 from ae arrived at 82.23
Patient 8 from ae waited for 0.00 hours, LOS: 37.82
Patient 9 from ae arrived at 86.42
Patient 9 from ae waited for 0.00 hours, LOS: 39.62
Patient 10 from ward arrived at 93.65
Patie

## 4 Review & Update:

The code provided by the LLM can run the simulation process successfully. The basic logic is correct, as it samples patients from different arrival sources, each with its own IAT and LOS distributions. The model includes the process of patient arrivals, waiting, their stay and departure, and prints out the messages of these events. However, there are several issues where the code does not fulfill the aim of the prompt. In the code, there are no specific generator function for different types of patients. Instead, the model uses one `patient_source` function and items from distribution dictionaries to sample varied admission types. Additionally, the code generated by LLM is relatively messy and hard to perceive, as it is not organised by classes.

To let the LLM code based on classes, We added additional instructions in the chatbox:

> Based on the prompt above, try re-organised the methods into two additional classes: (1) class `Patient` includes its `__init__` and `service` methods, within the service method it can request a bed, and stay in the CCU for a period sampling from LOS distributions. (2) class `CCU` contains its `__init__` and `arrival generator functions` for each type of patients (for instance, ae_arrivals_generator). Within the arrival generator function, it can sample a inter arrival time from IAT distributions, create a new patient of class Patient, and run the process by calling the service funtion.

The updated code and text are displayed below:

In [17]:
'''
To reorganize the simulation code according to your instructions, 
we'll create two new classes: Patient and CCU. 

The Patient class will manage individual patient characteristics 
and their stay in the CCU, including requesting a bed and the 
length of stay. 

The CCU class will manage the critical care unit resources and 
patient arrivals from different sources. 

This reorganization aims to encapsulate behaviors within 
appropriate classes and enhance the code's readability and maintainability.
'''
import simpy
import numpy as np
import itertools
from scipy.stats import lognorm

# Custom classes for distributions
class Exponential:
    def __init__(self, mean, random_seed=None):
        self.mean = mean
        self.rng = np.random.default_rng(seed=random_seed)
    
    def sample(self):
        return self.rng.exponential(self.mean)

class Lognormal:
    def __init__(self, mean, std, random_seed=None):
        self.mean = mean
        self.std = std
        self.rng = np.random.default_rng(seed=random_seed)
        self.sigma = np.sqrt(np.log(1 + (std/mean)**2))
        self.scale = np.exp(np.log(mean) - 0.5*self.sigma**2)
    
    def sample(self):
        return self.rng.lognormal(np.log(self.scale), self.sigma)

# Patient class
class Patient:
    def __init__(self, env, patient_id, source, los_dist):
        self.env = env
        self.patient_id = patient_id
        self.source = source
        self.los_dist = los_dist

    def service(self, ccu):
        arrive_time = self.env.now
        print(f"Patient {self.patient_id} from {self.source} arrived at {arrive_time:.2f}")

        with ccu.request() as request:
            yield request
            wait_time = self.env.now - arrive_time
            los = self.los_dist.sample()
            print(f"Patient {self.patient_id} from {self.source} waited for {wait_time:.2f} hours, LOS: {los:.2f}")
            yield self.env.timeout(los)
            print(f"Patient {self.patient_id} from {self.source} left at {self.env.now:.2f}")

# CCU class
class CCU:
    def __init__(self, env, capacity):
        self.env = env
        self.resource = simpy.Resource(env, capacity=capacity)

    def ae_arrivals_generator(self, iat_dist, los_dist):
        while True:
            yield self.env.timeout(iat_dist.sample())
            patient_id = next(patient_id_generator)
            patient = Patient(self.env, patient_id, "ae", los_dist)
            self.env.process(patient.service(self.resource))

    # Additional generators for other patient sources can be similarly defined
    def ward_arrivals_generator(self, iat_dist, los_dist):
        while True:
            yield self.env.timeout(iat_dist.sample())
            patient_id = next(patient_id_generator)
            patient = Patient(self.env, patient_id, "ward", los_dist)
            self.env.process(patient.service(self.resource))
            
    def emer_arrivals_generator(self, iat_dist, los_dist):
        while True:
            yield self.env.timeout(iat_dist.sample())
            patient_id = next(patient_id_generator)
            patient = Patient(self.env, patient_id, "emer", los_dist)
            self.env.process(patient.service(self.resource))
            
    def oth_arrivals_generator(self, iat_dist, los_dist):
        while True:
            yield self.env.timeout(iat_dist.sample())
            patient_id = next(patient_id_generator)
            patient = Patient(self.env, patient_id, "oth", los_dist)
            self.env.process(patient.service(self.resource))
            
    def xray_arrivals_generator(self, iat_dist, los_dist):
        while True:
            yield self.env.timeout(iat_dist.sample())
            patient_id = next(patient_id_generator)
            patient = Patient(self.env, patient_id, "xray", los_dist)
            self.env.process(patient.service(self.resource))

            
# Simulation parameters
RUN_LENGTH = 12 * 30 * 24  # 12 months
N_BEDS = 24
patient_id_generator = itertools.count()

# Initialize simulation environment
env = simpy.Environment()
ccu = CCU(env, N_BEDS)

# Patient arrival source distributions
IAT_DISTRIBUTIONS = {
    "ae": Exponential(22.72),
    # Add other sources here as needed
    "ward": Exponential(26.0),
    "emer": Exponential(37.0),
    "oth": Exponential(47.2),
    "xray": Exponential(575.0)
}

# Patient LOS distributions
LOS_DISTRIBUTIONS = {
    "ae": Lognormal(128.79, 267.51),
    # Add other sources here as needed
    "ward": Lognormal(177.89, 276.54),
    "emer": Lognormal(140.15, 218.02),
    "oth": Lognormal(212.86, 457.67),
    "xray": Lognormal(87.53, 108.15)
}


# Starting the patient sources for "ae"
env.process(ccu.ae_arrivals_generator(IAT_DISTRIBUTIONS["ae"], LOS_DISTRIBUTIONS["ae"]))
env.process(ccu.ward_arrivals_generator(IAT_DISTRIBUTIONS["ward"], LOS_DISTRIBUTIONS["ward"]))
env.process(ccu.emer_arrivals_generator(IAT_DISTRIBUTIONS["emer"], LOS_DISTRIBUTIONS["emer"]))
env.process(ccu.oth_arrivals_generator(IAT_DISTRIBUTIONS["oth"], LOS_DISTRIBUTIONS["oth"]))
env.process(ccu.xray_arrivals_generator(IAT_DISTRIBUTIONS["xray"], LOS_DISTRIBUTIONS["xray"]))

# Run the simulation
env.run(until=RUN_LENGTH)

Patient 0 from emer arrived at 5.62
Patient 0 from emer waited for 0.00 hours, LOS: 8.24
Patient 1 from oth arrived at 6.91
Patient 1 from oth waited for 0.00 hours, LOS: 50.22
Patient 0 from emer left at 13.86
Patient 2 from emer arrived at 20.73
Patient 2 from emer waited for 0.00 hours, LOS: 647.65
Patient 3 from oth arrived at 36.23
Patient 3 from oth waited for 0.00 hours, LOS: 186.67
Patient 4 from oth arrived at 36.53
Patient 4 from oth waited for 0.00 hours, LOS: 81.05
Patient 5 from ae arrived at 40.39
Patient 5 from ae waited for 0.00 hours, LOS: 26.34
Patient 6 from ae arrived at 43.54
Patient 6 from ae waited for 0.00 hours, LOS: 31.86
Patient 1 from oth left at 57.13
Patient 7 from ae arrived at 63.46
Patient 7 from ae waited for 0.00 hours, LOS: 31.18
Patient 5 from ae left at 66.73
Patient 8 from oth arrived at 70.29
Patient 8 from oth waited for 0.00 hours, LOS: 15.21
Patient 9 from emer arrived at 71.12
Patient 9 from emer waited for 0.00 hours, LOS: 53.81
Patient 6 fr

This time, the code quality has been improved dramatically, as we have two classes encapsulating these methods for running the process. Still, the model can be improved by creating a `class Scenario` to store the simulation inputs instead of using dictionaries. Thus, we manually updated the code.

In [18]:
import simpy
import numpy as np
import itertools
from distributions import Exponential, Lognormal


# Helper function to print out messages    
def trace(msg):
    '''
    Turning printing of events on and off.
    
    Params:
    -------
    msg: str
        string to print to screen.
    '''
    if TRACE:
        print(msg)

In [19]:
class Scenario:
    '''
    Parameter class for CCU simulation model inputs.
    '''
    def __init__(self):
        '''
        The init method sets up our defaults. 
        '''
        self.beds = simpy.Resource(env, capacity=N_BEDS)
        
        # Inter-arrival time (IAT) distributions for five types of patients
        self.ae_arrival_dist = Exponential(MEAN_IAT_ae, random_seed=SEEDS[0])
        self.ward_arrival_dist = Exponential(MEAN_IAT_ward, random_seed=SEEDS[1])
        self.emer_arrival_dist = Exponential(MEAN_IAT_emer, random_seed=SEEDS[2])
        self.oth_arrival_dist = Exponential(MEAN_IAT_oth, random_seed=SEEDS[3])
        self.xray_arrival_dist = Exponential(MEAN_IAT_xray, random_seed=SEEDS[4])

        # Length of stay (LOS) distributions for five types of patients
        self.ae_los_dist = Lognormal(MEAN_LOS_ae, STD_LOS_ae, random_seed=SEEDS[5])
        self.ward_los_dist = Lognormal(MEAN_LOS_ward, STD_LOS_ward, random_seed=SEEDS[6])
        self.emer_los_dist = Lognormal(MEAN_LOS_emer, STD_LOS_emer, random_seed=SEEDS[7])
        self.oth_los_dist = Lognormal(MEAN_LOS_oth, STD_LOS_oth, random_seed=SEEDS[8])
        self.xray_los_dist = Lognormal(MEAN_LOS_xray, STD_LOS_xray, random_seed=SEEDS[9])

In [20]:
class Patient:
    '''
    Patient in the CCU
    '''
    def __init__(self, identifier, env, source, args):
        '''
        Constructor method
        
        Params:
        -----
        identifier: int
            a numeric identifier for the patient.
            
        env: simpy.Environment
            the simulation environment
            
        args: Scenario
            The input data for the scenario
        '''
        # patient and environment
        self.identifier = identifier
        self.env = env        
        self.source = source        
        self.beds = args.beds
        

        # Length of stay (LOS) distributions for five types of patients
        self.ae_los_dist = args.ae_los_dist
        self.ward_los_dist = args.ward_los_dist
        self.emer_los_dist = args.emer_los_dist
        self.oth_los_dist = args.oth_los_dist
        self.xray_los_dist = args.xray_los_dist
        
        # individual parameter
        self.wait_time = 0.0


    def service(self):
        '''
        simulates the process for CCU 
        
        1. request and wait for a bed
        2. stay in CCU for a period of LOS
        3. exit system.
        
        '''
        # record the time that patient entered the system
        arrival_time = self.env.now

        # request a bed 
        with self.beds.request() as req:
            yield req
            
            # waiting time
            self.wait_time = self.env.now - arrival_time
            
            # sample LOS
            self.los = self.sample_los()
            trace(f'Patient {self.identifier} from {self.source} waited for {self.wait_time:.2f} hours. '\
                  + f'LOS: {self.los:.2f}')
            
            yield self.env.timeout(self.los)            
            
            trace(f'Patient {self.identifier} from {self.source} left at {env.now:.2f}')
            
            
    def sample_los(self):
        '''
        Sample the LOS distribution 
        according to different type of sources.
        '''
        if self.source == 'A&E':
            self.los = self.ae_los_dist.sample()
        elif self.source == 'Ward':
            self.los = self.ward_los_dist.sample()
        elif self.source == 'Emergency':
            self.los = self.emer_los_dist.sample()
        elif self.source == 'Other Hospital':
            self.los = self.oth_los_dist.sample()
        elif self.source == 'X-ray':
            self.los = self.xray_los_dist.sample()
                        
        return self.los

In [21]:
class CCU:  
    '''
    Model of a CCU
    '''
    def __init__(self, env, args):
        '''
        Contructor
        
        Params:
        -------
        env: simpy.Environment
        
        args: Scenario
            container class for simulation model inputs.
        '''
        self.env = env
        self.args = args
        
        self.ae_arrival_dist = args.ae_arrival_dist
        self.ward_arrival_dist = args.ward_arrival_dist
        self.emer_arrival_dist = args.emer_arrival_dist
        self.oth_arrival_dist = args.oth_arrival_dist
        self.xray_arrival_dist = args.xray_arrival_dist
        
        self.patients = []
        
            
    def ae_arrivals_generator(self):
        '''
        IAT generator for ae patients
        '''
        while True:
            inter_arrival_time = self.ae_arrival_dist.sample()
            yield self.env.timeout(inter_arrival_time)
            
            patient_count = next(identifier_generator)
            trace(f'Patient {patient_count} from A&E'\
                  + f' arrived at {self.env.now:.2f}')

            # create a new minor patient and pass in env and args
            new_patient = Patient(patient_count, self.env, 'A&E', self.args)
            
            # keep a record of the patient for results calculation
            self.patients.append(new_patient)
            
            # init the minor injury process for this patient
            self.env.process(new_patient.service())
            
            
    def ward_arrivals_generator(self):
        '''
        IAT generator for ward patients
        '''
        while True:
            inter_arrival_time = self.ward_arrival_dist.sample()
            yield self.env.timeout(inter_arrival_time)

            patient_count = next(identifier_generator)
            trace(f'Patient {patient_count} from Ward'\
                  + f' arrived at {self.env.now:.2f}')

            # create a new minor patient and pass in env and args
            new_patient = Patient(patient_count, self.env, 'Ward', self.args)
            
            # keep a record of the patient for results calculation
            self.patients.append(new_patient)
            
            # init the minor injury process for this patient
            self.env.process(new_patient.service())
            
            
    def emer_arrivals_generator(self):
        '''
        IAT generator for emergency patients
        '''
        while True:
            inter_arrival_time = self.emer_arrival_dist.sample()
            yield self.env.timeout(inter_arrival_time)

            patient_count = next(identifier_generator)
            trace(f'Patient {patient_count} from Emergency'\
                  + f' arrived at {self.env.now:.2f}')

            # create a new minor patient and pass in env and args
            new_patient = Patient(patient_count, self.env, 'Emergency', self.args)
            
            # keep a record of the patient for results calculation
            self.patients.append(new_patient)
            
            # init the minor injury process for this patient
            self.env.process(new_patient.service())
            
            
    def oth_arrivals_generator(self):
        '''
        IAT generator for other hospital patients
        '''
        while True:
            inter_arrival_time = self.oth_arrival_dist.sample()
            yield self.env.timeout(inter_arrival_time)

            patient_count = next(identifier_generator)
            trace(f'Patient {patient_count} from Other Hospital'\
                  + f' arrived at {self.env.now:.2f}')

            # create a new minor patient and pass in env and args
            new_patient = Patient(patient_count, self.env, 'Other Hospital', self.args)
            
            # keep a record of the patient for results calculation
            self.patients.append(new_patient)
            
            # init the minor injury process for this patient
            self.env.process(new_patient.service())
            
            
    def xray_arrivals_generator(self):
        '''
        IAT generator for xray patients
        '''
        while True:
            inter_arrival_time = self.xray_arrival_dist.sample()
            yield self.env.timeout(inter_arrival_time)

            patient_count = next(identifier_generator)
            trace(f'Patient {patient_count} from X-ray'\
                  + f' arrived at {self.env.now:.2f}')

            # create a new minor patient and pass in env and args
            new_patient = Patient(patient_count, self.env, 'X-ray', self.args)
            
            # keep a record of the patient for results calculation
            self.patients.append(new_patient)
            
            # init the minor injury process for this patient
            self.env.process(new_patient.service())

In [22]:
# Run the simulation model
########### Simulation parameters ####################
N_BEDS = 24

# Patient inter-arrival time (IAT) distributions
MEAN_IAT_ae = 22.72
MEAN_IAT_ward = 26.0
MEAN_IAT_emer = 37.0
MEAN_IAT_oth = 47.2
MEAN_IAT_xray = 575.0

# Patient length of stay (LOS) distributions
MEAN_LOS_ae = 128.79
STD_LOS_ae = 267.51
MEAN_LOS_ward = 177.89 
STD_LOS_ward = 276.54
MEAN_LOS_emer = 140.15 
STD_LOS_emer = 218.02
MEAN_LOS_oth = 212.86
STD_LOS_oth = 457.67
MEAN_LOS_xray = 87.53
STD_LOS_xray = 108.15

# SEEDS to reproduce results of a single run
REPRODUCIBLE_RUN = True    
if REPRODUCIBLE_RUN:
    SEEDS = [42, 43, 44, 45, 46, 47, 48, 49, 50, 51]
else:
    SEEDS = [None, None, None, None, None, None, None, None, None, None]
    
# RUN lENGTH
RUN_LENGTH = 12 * 30 * 24  # 12 months

# Turn off tracing
TRACE = True
##########################################################

# generate patient identifier
identifier_generator = itertools.count()

# create simpy environment
env = simpy.Environment()

# base case scenario with default parameters
default_args = Scenario()


# create the model
model = CCU(env, default_args)

# setup the process
env.process(model.ae_arrivals_generator())
env.process(model.ward_arrivals_generator())
env.process(model.emer_arrivals_generator())
env.process(model.oth_arrivals_generator())
env.process(model.xray_arrivals_generator())


env.run(until=RUN_LENGTH)
print(f'End of run. simulation clock time = {env.now}')


Patient 0 from Ward arrived at 4.30
Patient 0 from Ward waited for 0.00 hours. LOS: 68.43
Patient 1 from Ward arrived at 6.54
Patient 1 from Ward waited for 0.00 hours. LOS: 429.57
Patient 2 from Ward arrived at 7.93
Patient 2 from Ward waited for 0.00 hours. LOS: 119.17
Patient 3 from Emergency arrived at 13.71
Patient 3 from Emergency waited for 0.00 hours. LOS: 142.69
Patient 4 from Emergency arrived at 31.66
Patient 4 from Emergency waited for 0.00 hours. LOS: 351.09
Patient 5 from Other Hospital arrived at 35.95
Patient 5 from Other Hospital waited for 0.00 hours. LOS: 170.10
Patient 6 from Ward arrived at 38.90
Patient 6 from Ward waited for 0.00 hours. LOS: 457.57
Patient 7 from A&E arrived at 54.62
Patient 7 from A&E waited for 0.00 hours. LOS: 23.61
Patient 0 from Ward left at 72.73
Patient 8 from Emergency arrived at 76.27
Patient 8 from Emergency waited for 0.00 hours. LOS: 98.77
Patient 9 from Other Hospital arrived at 77.00
Patient 9 from Other Hospital waited for 0.00 hou

## 5 Performance metrics:

In this section, we will create a `class Auditor` and calculate the outcome metrics: mean waiting time, mean queue length, 
mean number of occupied beds, and occupancy percentage, etc.

In [23]:
import pandas as pd

class Auditor:
    def __init__(self, env, run_length, bed_counts, first_obs=None, interval=None):
        '''
        Auditor Constructor
        
        Params:
        -----
        env: simpy.Environment
            
        first_obs: float, optional (default=None)
            Time of first scheduled observation.  If none then no scheduled
            audit will take place
        
        interval: float, optional (default=None)
            Time period between scheduled observations. If none then no scheduled
            audit will take place
        '''
        self.env = env
        self.first_observation = first_obs
        self.interval = interval
        self.run_length = run_length
        self.bed_counts = bed_counts
        
        self.queues = []
        self.services = []
        
        # dict to hold states
        self.metrics = {}
        
        # scheduled the periodic audits
        if not first_obs is None:
            env.process(self.scheduled_observation())
            env.process(self.process_end_of_run())
            
    def add_resource_to_audit(self, resource, name, audit_type='qs'):
        if 'q' in audit_type:
            self.queues.append((name, resource))
            self.metrics[f'queue_length_{name}'] = []
        
        if 's' in audit_type:
            self.services.append((name, resource))
            self.metrics[f'occupied_{name}'] = []   
                    
    def record_queue_length(self):
        for name, res in self.queues:
            self.metrics[f'queue_length_{name}'].append(len(res.queue)) 
               
    def record_occupied_bed(self):
        for name, res in self.services:
            self.metrics[f'occupied_{name}'].append(res.count) 

            
    def scheduled_observation(self):
        '''
        simpy process to control the frequency of 
        auditor observations of the model.  
        
        The first observation takes place at self.first_obs
        and subsequent observations are spaced self.interval
        apart in time.
        '''
        # delay first observation
        yield self.env.timeout(self.first_observation)
        self.record_queue_length()
        self.record_occupied_bed()
        
        while True:
            yield self.env.timeout(self.interval)
            self.record_queue_length()
            self.record_occupied_bed()
               
        
    def process_end_of_run(self):
        '''
        Create an end of run summary
        
        Returns:
        ---------
            pd.DataFrame
        '''
        
        yield self.env.timeout(self.run_length - 1)
        
        run_results = {}

        for name, res in self.queues:
            queue_length = np.array(self.metrics[f'queue_length_{name}'])
            run_results[f'mean_queue_{name}'] = queue_length.mean()
            
        for name, res in self.services:
            serviced_beds = np.array(self.metrics[f'occupied_{name}'])
            run_results[f'mean_occupied_{name}'] = serviced_beds.mean()
            run_results[f'occupancy_rate_{name}'] = (serviced_beds.mean() / self.bed_counts) 

        self.summary_frame = pd.Series(run_results).to_frame()
        self.summary_frame.columns = ['estimate']        

In [24]:
# collect performance metrics
def run_results(model, auditor):
    df_results = auditor.summary_frame
    
    # total admissions
    total_admissions = len(model.patients)
    ae_admissions = sum(patient.source == 'A&E' for patient in model.patients)
    ward_admissions = sum(patient.source == 'Ward' for patient in model.patients)
    emer_admissions = sum(patient.source == 'Emergency' for patient in model.patients)
    oth_admissions = sum(patient.source == 'Other Hospital' for patient in model.patients)
    xray_admissions = sum(patient.source == 'X-ray' for patient in model.patients)
        
    # waiting time = sum(waiting times) / no. patients
    mean_wait_time = np.array([patient.wait_time 
                                for patient in model.patients]).mean()
    
    # bed days utilisation = sum(los) / (run length X no. beds)
    bed_day_util = np.array([patient.los 
                     for patient in model.patients]).sum() / \
                    (RUN_LENGTH * N_BEDS)

    # append to results df
    new_row = pd.DataFrame({'estimate':{'total_admissions': total_admissions,
                                        'A&E_admissions': ae_admissions,
                                        'Ward_admissions': ward_admissions,
                                        'Emergency_admissions': emer_admissions,
                                        'Other_hospital_admissions': oth_admissions,
                                        'Xray_admissions': xray_admissions,
                                        'mean_wait_hours': mean_wait_time, 
                                        'bed_days_util': bed_day_util}})

    df_results = pd.concat([df_results, new_row])
    return df_results

In [25]:
# Run the simulation model
# RUN lENGTH
RUN_LENGTH = 12 * 30 * 24  # 12 months

# Turn off tracing
TRACE = False

# scheduled audit intervals in minutes.
FIRST_OBS = 24
OBS_INTERVAL = 48
N_BEDS = 24

# generate patient identifier
identifier_generator = itertools.count()

# create simpy environment
env = simpy.Environment()

# base case scenario with default parameters
default_args = Scenario()

# instantiate an auditor
auditor = Auditor(env, RUN_LENGTH, N_BEDS, FIRST_OBS, OBS_INTERVAL)
auditor.add_resource_to_audit(default_args.beds, 'beds')

# create the model
model = CCU(env, default_args)

# setup the process
env.process(model.ae_arrivals_generator())
env.process(model.ward_arrivals_generator())
env.process(model.emer_arrivals_generator())
env.process(model.oth_arrivals_generator())
env.process(model.xray_arrivals_generator())


env.run(until=RUN_LENGTH)
print(f'End of run. simulation clock time = {env.now}')


print('\nSingle run results\n------------------')
run_results(model, auditor).round(2)

End of run. simulation clock time = 8640

Single run results
------------------


,estimate
mean_queue_beds,1.13
mean_occupied_beds,20.38
occupancy_rate_beds,0.85
A&E_admissions,387.00
Emergency_admissions,254.00
Other_hospital_admissions,186.00
Ward_admissions,321.00
Xray_admissions,19.00
bed_days_util,0.89
mean_wait_hours,8.33


In [26]:
# print the annual admissions stratified by source in the paper
# data were collected over a 3-year period

print(f'A&E admissions: {int(1134/3)}')
print(f'Ward admissions: {int(916/3)}')
print(f'Emergency admissions: {int(715/3)}')
print(f'Other hospital admissions: {int(235/3)}')
print(f'Xray admissions: {int(44/3)}')

A&E admissions: 378
Ward admissions: 305
Emergency admissions: 238
Other hospital admissions: 78
Xray admissions: 14


We have gathered the single-run results from unplanned admissions. As shown in the result table, the mean number occupied was 20.38 in simulation, compared with 20.10 according to the data reported in the paper. The total admission count was 1167 in simulation, compared with 1359 according to the data, as we did not include elective patients yet. The admissions counts stratified by source were similar compared to the data reported in the paper except for admissions from other hospitals. In the next iteration of the model, the code will be updated to incorporate elective surgery patients into the current simulation model. 

# End